## Import & Install

In [ ]:
#!pip install -r requirements.txt

In [ ]:
import pandas as pd
import random
import os
import numpy as np
import warnings
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from datetime import datetime

from sklearn.preprocessing import RobustScaler, PowerTransformer

from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Data Load

In [ ]:
###  INPUT ###
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')

## Data Pre-processing

In [ ]:
input_data = input_data.drop(columns=['frmDist'])

# Split the data into training and testing sets
X = input_data[input_data.drop(columns=['outtrn_cumsum','HeatingEnergyUsage_cumsum']).columns]
Y = input_data[['outtrn_cumsum','HeatingEnergyUsage_cumsum']]

In [ ]:
X.columns = X.columns.str.replace(' ', '_')

In [ ]:
value_counts_per_column = X.nunique()
multi_value_columns = value_counts_per_column[value_counts_per_column > 1].index
X = X[multi_value_columns]

In [ ]:
X = X.drop(columns=['frmYear', 'frmWeek'])

## Scaling

In [ ]:
num_features = X.columns[X.columns != 'date']

In [ ]:
scaler = PowerTransformer()
X[num_features] = scaler.fit_transform(X[num_features])

## Feature Selection

In [ ]:
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [ ]:
X_1 = X.drop(columns=['tcdmt',
                      'WaterUsage',
                      'WaterCost',
                      'FertilizerUsage',
                      'FertilizerCost',
                      'CO2Usage',
                      'CO2Cost',
                      'MistUsageTime',
                      'Mist_Cost'])

In [ ]:
X_2 = X.drop(columns=['hvstCo'])

## Add time variable

In [ ]:
date = pd.to_datetime(X_1['date'], format='%Y%m%d')
X_1.loc[:, 'year'] = date.dt.year
X_1.loc[:, 'month'] = date.dt.month
X_1.loc[:, 'week'] = date.dt.isocalendar().week.astype(np.int32)
X_1.loc[:, 'day'] = date.dt.weekday

In [ ]:
date = pd.to_datetime(X_2['date'], format='%Y%m%d')
X_2.loc[:, 'year'] = date.dt.year
X_2.loc[:, 'month'] = date.dt.month
X_2.loc[:, 'week'] = date.dt.isocalendar().week.astype(np.int32)
X_2.loc[:, 'day'] = date.dt.weekday

In [ ]:
X_1 = X_1.drop(columns='date')
X_2 = X_2.drop(columns='date')

## Train_test_split

In [ ]:
X_train_01, X_test_01, y_train_01, y_test_01 = train_test_split(X_1, Y['outtrn_cumsum'], test_size=0.2, random_state=42)
X_train_02, X_test_02, y_train_02, y_test_02 = train_test_split(X_2, Y['HeatingEnergyUsage_cumsum'], test_size=0.2, random_state=42)

In [ ]:
X_train_01 = X_train_01.reset_index(drop=True)
X_test_01 = X_test_01.reset_index(drop=True)
y_train_01 = y_train_01.reset_index(drop=True)
y_test_01 = y_test_01.reset_index(drop=True)

In [ ]:
X_train_02 = X_train_02.reset_index(drop=True)
X_test_02 = X_test_02.reset_index(drop=True)
y_train_02 = y_train_02.reset_index(drop=True)
y_test_02 = y_test_02.reset_index(drop=True)

## Modeling

In [ ]:
# Calculate RMSE between the predictions and actual 'y' values
def calculate_rmse(targets, predictions):
    """
    Calculate the Root Mean Squared Error (RMSE) between predicted and target values.

    :param predictions: Predicted values.
    :type predictions: array-like
    :param targets: Target values.
    :type targets: array-like
    :return: RMSE value.
    :rtype: float
    """
    from sklearn.metrics import mean_squared_error
    return np.sqrt(mean_squared_error(targets, predictions))


# Calculate r2_score between the predictions and actual 'y' values
def calculate_R2_score(y_test,y_pred):
    from sklearn.metrics import r2_score
    return r2_score(y_test, y_pred)

## Lgbm

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

In [ ]:
# Define hyperparameters and their respective search ranges
param_ranges = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [31, 127, 255],
    'max_depth': [8, 12, 16],
    'subsample': [0.5, 0.75, 1.0],
    'colsample_bytree': [0.5, 0.75, 1.0],
    'reg_alpha': [0.0, 0.5, 1.0],
    'reg_lambda': [0.0, 0.5, 1.0]
}

best_rmse = float('inf')
best_params = {}

for params in product(*param_ranges.values()):
    params_dict = dict(zip(param_ranges.keys(), params))
    params_dict['n_jobs'] = -1
    params_dict['random_state'] = 42

    model = LGBMRegressor(**params_dict)
    model.fit(X_train_01, y_train_01)
    y_pred_01 = model.predict(X_test_01)

    rmse = np.sqrt(mean_squared_error(y_test_01, y_pred_01))

    if rmse < best_rmse:
        best_rmse = rmse
        best_params = params_dict

print('Best RMSE:', best_rmse)
print('Best params:', best_params)

NameError: name 'product' is not defined

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 31, 255),
        'max_depth': trial.suggest_int('max_depth', 8, 16),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'n_jobs' : -1,
        'random_state': 42
    }

    model = LGBMRegressor(**params)
    model.fit(X_train_01, y_train_01)
    y_pred_01 = model.predict(X_test_01)

    rmse = calculate_rmse(y_test_01, y_pred_01)
    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

[I 2023-09-17 09:48:16,834] A new study created in memory with name: no-name-667a2309-b6e3-4f53-8092-1d20b48e6130


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003581 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5706
[LightGBM] [Info] Number of data points in the train set: 67872, number of used features: 28
[LightGBM] [Info] Start training from score 24115.907462
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

[I 2023-09-17 09:48:22,272] Trial 0 finished with value: 1549.0389455199397 and parameters: {'n_estimators': 747, 'learning_rate': 0.08954959644463323, 'num_leaves': 110, 'max_depth': 12, 'subsample': 0.9511703548462607, 'colsample_bytree': 0.9156346562199374, 'reg_alpha': 0.6473370348303366, 'reg_lambda': 0.40514459210596454}. Best is trial 0 with value: 1549.0389455199397.


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003678 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5706
[LightGBM] [Info] Number of data points in the train set: 67872, number of used features: 28
[LightGBM] [Info] Start training from score 24115.907462


[I 2023-09-17 09:48:23,927] Trial 1 finished with value: 1798.5785756827636 and parameters: {'n_estimators': 420, 'learning_rate': 0.07390890549938685, 'num_leaves': 49, 'max_depth': 14, 'subsample': 0.6202178386669435, 'colsample_bytree': 0.7149051917718406, 'reg_alpha': 0.5961688705550098, 'reg_lambda': 0.2561662696361826}. Best is trial 0 with value: 1549.0389455199397.


...

[I 2023-09-17 09:59:56,185] Trial 297 finished with value: 1500.1354432442606 and parameters: {'n_estimators': 143, 'learning_rate': 0.03831293834755938, 'num_leaves': 243, 'max_depth': 16, 'subsample': 0.9633338131970576, 'colsample_bytree': 0.9386677103957995, 'reg_alpha': 0.4248755773915801, 'reg_lambda': 0.40294734935999577}. Best is trial 272 with value: 1492.694041328624.


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003582 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5706
[LightGBM] [Info] Number of data points in the train set: 67872, number of used features: 28
[LightGBM] [Info] Start training from score 24115.907462


[I 2023-09-17 09:59:58,326] Trial 298 finished with value: 1608.6269654479217 and parameters: {'n_estimators': 166, 'learning_rate': 0.03531150101619765, 'num_leaves': 237, 'max_depth': 16, 'subsample': 0.9739715496158821, 'colsample_bytree': 0.8663578340217066, 'reg_alpha': 0.17481655947703775, 'reg_lambda': 0.5297991942079006}. Best is trial 272 with value: 1492.694041328624.


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003768 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5706
[LightGBM] [Info] Number of data points in the train set: 67872, number of used features: 28
[LightGBM] [Info] Start training from score 24115.907462


[I 2023-09-17 09:59:59,880] Trial 299 finished with value: 1820.9849061584048 and parameters: {'n_estimators': 102, 'learning_rate': 0.036739833580164125, 'num_leaves': 252, 'max_depth': 16, 'subsample': 0.9506559529284113, 'colsample_bytree': 0.8938476577606786, 'reg_alpha': 0.280686854375069, 'reg_lambda': 0.3898407719108142}. Best is trial 272 with value: 1492.694041328624.


In [ ]:
best_params = study.best_params
print("Best parameters found: ", best_params)

Best parameters found:  {'n_estimators': 156, 'learning_rate': 0.03715433703760937, 'num_leaves': 248, 'max_depth': 16, 'subsample': 0.9884963091311882, 'colsample_bytree': 0.9263792221113196, 'reg_alpha': 0.7692472328343856, 'reg_lambda': 0.38355777389286627}


In [ ]:
params = {'n_estimators': 156, 'learning_rate': 0.03715433703760937, 'num_leaves': 248, 'max_depth': 16, 'subsample': 0.9884963091311882, 'colsample_bytree': 0.9263792221113196, 'reg_alpha': 0.7692472328343856, 'reg_lambda': 0.38355777389286627, 'random_state' : 42}

In [ ]:
model = LGBMRegressor(**params)
model.fit(X_train_01, y_train_01)
y_pred_01 = model.predict(X_test_01)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011355 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5706
[LightGBM] [Info] Number of data points in the train set: 67872, number of used features: 28
[LightGBM] [Info] Start training from score 24115.907462


In [ ]:
rmse = calculate_rmse(y_test_01, y_pred_01)
print("RMSE:", rmse)

RMSE: 1492.694041328624


In [ ]:
"""
param_grid = {
    'max_depth': list(range(8, 16)),
    'num_leaves': [64, 127, 255]
}

# Create the LightGBM model
model = LGBMRegressor(random_state=42)

# Create Grid Search with RMSE as the scoring metric
grid_search = GridSearchCV(model, param_grid, cv=6, scoring='neg_mean_squared_error', verbose=1)

# Fit the Grid Search to the data
grid_search.fit(X_train_01, y_train_01)

# Print the best parameters and corresponding RMSE
print("Best Parameters: ", grid_search.best_params_)
print("Best RMSE: ", np.sqrt(-grid_search.best_score_))
"""

Fitting 6 folds for each of 24 candidates, totalling 144 fits
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010455 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5706
[LightGBM] [Info] Number of data points in the train set: 56560, number of used features: 28
[LightGBM] [Info] Start training from score 24121.918345
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007573 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5706
[LightGBM] [Info] Number of data points in the train set: 56560, number of used features: 28
[LightGBM] [Info] Start training from score 24162.047925
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005384 seconds.
You can set `force_row_wise=true` to remove the overhead.


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008361 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5706
[LightGBM] [Info] Number of data points in the train set: 56560, number of used features: 28
[LightGBM] [Info] Start training from score 24052.513917
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010086 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5706
[LightGBM] [Info] Number of data points in the train set: 56560, number of used features: 28
[LightGBM] [Info] Start training 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008982 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5706
[LightGBM] [Info] Number of data points in the train set: 56560, number of used features: 28
[LightGBM] [Info] Start training from score 24162.047925
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009422 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5706
[LightGBM] [Info] Number of data points in the train set: 56560, number of used features: 28
[LightGBM] [Info] Start training from score 24158.564945
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008509 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5706
[LightGBM] [Info] Number of data points in the train set: 56560, number of used features: 28
[LightGBM] [Info] Start training from score 23990.678414
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008859 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5706
[LightGBM] [Info] Number of 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010745 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5706
[LightGBM] [Info] Number of data points in the train set: 56560, number of used features: 28
[LightGBM] [Info] Start training from score 23990.678414
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

KeyboardInterrupt: 

In [ ]:
# num_leaves = 31
for i in range(19):
    print(i+2, rmse_score[i])

2 14927.543482419429
3 8242.174048698133
4 4419.776382636725
5 2427.86127972125
6 1896.645858240101
7 1791.9637569679112
8 1647.7350044910972
9 1635.6743358595968
10 1622.7270931181206
11 1615.457786617537
12 1619.8491543817902
13 1609.100322615947
14 1616.6940297896851
15 1625.6565674538435
16 1616.8317955013345
17 1614.3526590538534
18 1622.575783577075
19 1618.381068843692
20 1618.3810688436918


In [ ]:
# num_leaves = 63
for i in range(19):
    print(i+2, rmse_score[i])

2 14927.543482419427
3 8242.174048698134
4 4419.776382636751
5 2392.845700800638
6 1803.0488070653353
7 1650.2453805556427
8 1563.4734323466946
9 1558.4618410116327
10 1557.0146780502732
11 1555.6838992974594
12 1561.5328554417638
13 1563.957946566221
14 1557.4393388438375
15 1560.028095396178
16 1564.115127614212
17 1554.0101209375061
18 1561.4656911132984
19 1558.0587786198532
20 1561.7482089882221


In [ ]:
# num_leaves = 127
for i in range(19):
    print(i+2, rmse_score[i])

2 14927.543482419429
3 8242.174048698134
4 4419.776382636751
5 2392.845700800638
6 1803.0477522423446
7 1636.4181774922865
8 1545.911325131294
9 1539.5491966666889
10 1529.982794090917
11 1531.7650005195483
12 1519.5637507404736
13 1532.6892775657734
14 1529.9029591399205
15 1530.8720446755738
16 1534.7769333010765
17 1527.2451643054503
18 1532.5233419853962
19 1531.4924306604983
20 1531.0129012576747


In [ ]:
# num_leaves = 255
for i in range(19):
    print(i+2, rmse_score[i])

2 14927.543482419427
3 8242.174048698134
4 4419.776382636751
5 2392.8457008006503
6 1803.047752242346
7 1636.418177492287
8 1552.8836767257433
9 1544.5662625656123
10 1523.0700337193439
11 1522.6872189431713
12 1533.4535377737886
13 1548.9997241203548
14 1540.5061174857672
15 1541.7481220397488
16 1547.3388742718228
17 1542.3505934496952
18 1547.5444963632904
19 1542.3630484883918
20 1541.7281982463699


In [ ]:
rmse_score = []
for i in range(2,21):
    model = LGBMRegressor(max_depth=i, num_leaves=255, random_state=42)
    model.fit(X_train_02, y_train_02)
    y_pred = model.predict(X_test_02)
    rmse = calculate_rmse(y_test_02, y_pred)
    rmse_score.append(rmse)
    print(i, '번째 RMSE :', rmse)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017448 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6990
[LightGBM] [Info] Number of data points in the train set: 67872, number of used features: 36
[LightGBM] [Info] Start training from score 135668.753849
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
2 번째 RMSE : 198256.6085480426
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016838 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6990
[LightGBM] [Info] Number of data points in the train set: 67872, number of used features: 36
[LightGBM] [Info] Start training from score 135668.753849
[LightGBM] [Warning] No further splits with p

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
14 번째 RMSE : 55101.27720195445
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016081 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6990
[LightGBM] [Info] Number of data points in the train set: 67872, number of used features: 36
[LightGBM] [Info] Start training from score 135668.753849
[LightGBM] [Warning

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
17 번째 RMSE : 54326.16279550224
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019165 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6990
[LightGBM] [Info] Number of data points in the train set: 67872, number of used features: 36
[LightGBM] [Info] Start training from score 135668.753849
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [ ]:
# num_leaves = 31
for i in range(19):
    print(i+2, rmse_score[i])

2 198256.6085480426
3 145675.99403084596
4 111254.74065248352
5 80984.55842363233
6 69031.0525261168
7 61527.17266513621
8 59765.501098986584
9 58355.95359260089
10 59205.881807366495
11 57536.64501246298
12 56591.8830350745
13 57049.798506822
14 56859.28478947172
15 56775.18350777687
16 56597.1467834299
17 56542.9560601135
18 56542.9560601135
19 56542.9560601135
20 56542.9560601135


In [ ]:
# num_leaves = 63
for i in range(19):
    print(i+2, rmse_score[i])

2 198256.6085480426
3 145675.99403084596
4 111254.74065248352
5 80984.55840887537
6 67726.85326451024
7 58858.96426587213
8 54950.933475485675
9 54562.01260023598
10 55283.01750396874
11 56007.048428822425
12 54033.72190942173
13 56761.12877734198
14 56292.378578358854
15 56044.858624010274
16 56088.94467931271
17 56155.14831986383
18 55816.57495317861
19 55589.494711470514
20 55746.47355779422


In [ ]:
# num_leaves = 127
for i in range(19):
    print(i+2, rmse_score[i])

2 198256.6085480426
3 145675.99403084596
4 111254.74065248352
5 80984.55840887537
6 67726.85326451024
7 60700.11526516629
8 54040.49379325516
9 55539.30121858249
10 55053.040567463635
11 55365.458469679834
12 54357.875465678335
13 55233.263937049225
14 54959.85092663698
15 53984.70743144585
16 53878.09180447729
17 53840.2069096456
18 53763.16095462596
19 53316.04071973877
20 53668.96162420824


In [ ]:
# num_leaves = 255
for i in range(19):
    print(i+2, rmse_score[i])

2 198256.6085480426
3 145675.99403084596
4 111254.74065248352
5 80984.55840887537
6 67726.85326451024
7 60700.11526516629
8 54040.49379325516
9 54098.32319601251
10 54777.99009795554
11 54807.32294233584
12 54394.660792848284
13 55106.80964322946
14 55101.27720195445
15 54308.919076127284
16 54293.86150620497
17 54326.16279550224
18 54162.69859623196
19 54541.62573777988
20 54065.78116638572


In [ ]:
y_test = pd.concat([y_test_01, y_test_02], axis=1)

In [ ]:
y_pred_concatenated = np.column_stack((y_pred_01, y_pred_02))
y_pred = pd.DataFrame(y_pred_concatenated, columns=['y_pred_01', 'y_pred_02'])

In [ ]:
rmse = calculate_rmse(y_test, y_pred)
r2score = calculate_R2_score(y_test, y_pred)

In [ ]:
### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)

In [ ]:
y_pred